In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/Documents/git_workspace/16715/dynamics-simulation-leg/scripts/Project.toml`


In [2]:
using RigidBodyDynamics
using LinearAlgebra
using MeshCatMechanisms
using MeshCat
using StaticArrays
using SparseArrays
using ForwardDiff

WebIO._IJuliaInit()

In [3]:
# link lengths
const l0 = [0.1; 0; 0]
const l1 = [0.3; 0; 0]
const l2 = [0.3; 0; 0]
const l3 = 0.1
const l4 = 0.2
const l5 = 0.0205
const lc = [l3; 0; 0]
const lee = [l3 + l4; 0; l5] # sqrt((l3 + l4)^2 + l5^2)

# CoM locations
const l_cb = [0; 0.004; 0]
const l_c0 = [0.0125108364230515; 0.00117191218927888; 0]
const l_c1 = [0.149359714867044; 0; 0]
const l_c2 = [0.0469412900551914; 0; 0]
const l_c3 = [0.113177000131857; 0; -0.015332867880069]

# link masses
const mb = 7  # kg
const m0 = 0.24644240965487
const m1 = 0.0707939028219395
const m2 = 0.276735496985514
const m3 = 0.130824780046739
# const m = Diagonal([m0, m1, m2, m3])
    
# gravity, obviously
const g = 9.807

# mass moment of inertia in axis of rotation
const Ib = Array([0.0024241 5.252E-06 2.0733E-19; 
                  5.252E-06 0.0044176 -3.1153E-19; 
                  2.0733E-19 -3.1153E-19 0.0022481])

const I0 = Array([3.83120149546952E-05 1.46925714738609E-05 -8.60106401672571E-06;
                  1.46925714738609E-05 0.000172067745507247 1.0427260925207E-06;
                  -8.60106401672571E-06 1.0427260925207E-06 0.00014745218068435])

const I1 = Array([3.06999775886187E-06 7.91090301514898E-12 -1.43705963146176E-12;
                  7.91090301514898E-12 0.000147960574744097 1.30742394049546E-11;
                  -1.43705963146176E-12 1.30742394049546E-11 0.000147884231885009])

const I2 = Array([3.43038397803592E-05 -2.90339844227483E-07 6.18680397558952E-06;
                  -2.90339844227483E-07 0.000302324068012293 2.25016327583562E-08;
                  6.18680397558952E-06 2.25016327583562E-08 0.00028292376778719])

const I3 = Array([1.76996970020568E-05 -5.3695427116208E-07 7.62350214406387E-07;
                  -5.3695427116208E-07 0.000164188445564489 -2.77843753828047E-07;
                  7.62350214406387E-07 -2.77843753828047E-07 0.000160656046697151])

3×3 Matrix{Float64}:
  1.76997e-5  -5.36954e-7    7.6235e-7
 -5.36954e-7   0.000164188  -2.77844e-7
  7.6235e-7   -2.77844e-7    0.000160656

In [4]:
M̄ = [mb*I(3) zeros(3, 27)
     zeros(3,3) Ib zeros(3, 24)
     zeros(3,6) m0*I(3) zeros(3,21);
     zeros(3,9) I0 zeros(3,18);
     zeros(3,12) m1*I(3) zeros(3,15);
     zeros(3,15) I1 zeros(3, 12);
     zeros(3,18) m2*I(3) zeros(3, 9);
     zeros(3, 21) I2 zeros(3, 6);
     zeros(3, 24) m3*I(3) zeros(3, 3);
     zeros(3, 27) I3]

30×30 SparseMatrixCSC{Float64, Int64} with 60 stored entries:
⠑⢄⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠘⠛⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⢱⣶⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠑⢄⣀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠘⠛⢄⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢱⣶⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⣀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠛

In [5]:
function hat(ω)
    return [0 -ω[3] ω[2];
            ω[3] 0 -ω[1];
            -ω[2] ω[1] 0]
end

function L(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I + hat(Q[2:4])]
end

function R(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I - hat(Q[2:4])]
end

H = [zeros(1,3); I];

T = Diagonal([1.0; -1; -1; -1])

function G(Q)
    return L(Q)*H  # 4x3
end

function Ḡ(q)
    Qb = q[4:7]
    Q0 = q[11:14]
    Q1 = q[18:21]
    Q2 = q[25:28]
    Q3 = q[32:35]
    
    # [I(3) zeros(3,4); zeros(4,3) G(Q) zeros(4,1); zeros(1,6) 1]  # 8x7
    return blockdiag(sparse(I, 3, 3), sparse(G(Qb)), 
                     sparse(I, 3, 3), sparse(G(Q0)), 
                     sparse(I, 3, 3), sparse(G(Q1)),
                     sparse(I, 3, 3), sparse(G(Q2)),
                     sparse(I, 3, 3), sparse(G(Q3)),)
end

Ḡ (generic function with 1 method)

In [6]:
function Expq(ϕ)
    
    # The quaternion exponential map ϕ → q 
    q = zeros(4)
    θ = norm(ϕ)
    q = [cos(θ/2); 0.5*ϕ*sinc(θ/(2*pi))]
    
    return q
end

function rotate(Q, p)
    # Rotate a position vector p by a quaternion Q
    return H'L(Q)*R(Q)'*H*p
end

rotate (generic function with 1 method)

In [7]:
#Input Jacobian
function B(q)
    #[zeros(7); 1.0; zeros(2)]
    I(10)
end

B (generic function with 1 method)

In [8]:
function del(m, I, r1, r2, r3, Q1, Q2, Q3)
    [(1/h)*m*(r2-r1) - (1/h)*m*(r3-r2);
     (2.0/h)*G(Q2)'*L(Q1)*H*I*H'*L(Q1)'*Q2 + (2.0/h)*G(Q2)'*T*R(Q3)'*H*I*H'*L(Q2)'*Q3]
end
    
    
function DEL(q_1,q_2,q_3,λ,F1,F2)
    
    rb_1 = q_1[1:3]
    Qb_1 = q_1[4:7]
    r0_1 = q_1[8:10]
    Q0_1 = q_1[11:14]
    r1_1 = q_1[15:17]
    Q1_1 = q_1[18:21]
    r2_1 = q_1[22:24]
    Q2_1 = q_1[25:28]
    r3_1 = q_1[29:31]
    Q3_1 = q_1[32:35]
    
    rb_2 = q_2[1:3]
    Qb_2 = q_2[4:7]
    r0_2 = q_2[8:10]
    Q0_2 = q_2[11:14]
    r1_2 = q_2[15:17]
    Q1_2 = q_2[18:21]
    r2_2 = q_2[22:24]
    Q2_2 = q_2[25:28]
    r3_2 = q_2[29:31]
    Q3_2 = q_2[32:35]
    
    rb_3 = q_3[1:3]
    Qb_3 = q_3[4:7]
    r0_3 = q_3[8:10]
    Q0_3 = q_3[11:14]
    r1_3 = q_3[15:17]
    Q1_3 = q_3[18:21]
    r2_3 = q_3[22:24]
    Q2_3 = q_3[25:28]
    r3_3 = q_3[29:31]
    Q3_3 = q_3[32:35]

    del1 = [del(mb, Ib, rb_1, rb_2, rb_3, Qb_1, Qb_2, Qb_3);
            del(m0, I0, r0_1, r0_2, r0_3, Q0_1, Q0_2, Q0_3);
            del(m1, I1, r1_1, r1_2, r1_3, Q1_1, Q1_2, Q1_3);
            del(m2, I2, r2_1, r2_2, r2_3, Q2_1, Q2_2, Q2_3);
            del(m3, I3, r3_1, r3_2, r3_3, Q3_1, Q3_2, Q3_3)] 

    return del1 + (h/2.0)*F1 + (h/2.0)*F2 + h*Dc(q_2)'*λ
end

DEL (generic function with 1 method)

In [9]:
function Dq3DEL(q_1,q_2,q_3,λ,F1,F2)
    # @show Ḡ(q_3)
    ForwardDiff.jacobian(dq->DEL(q_1,q_2,dq,λ,F1,F2), q_3)*Ḡ(q_3)
end

Dq3DEL (generic function with 1 method)

In [39]:
function c(q)
    # constraint function
    # c(q_0) should be all zeros
    rb = q[1:3]
    Qb = q[4:7]
    r0 = q[8:10]
    Q0 = q[11:14]
    r1 = q[15:17]
    Q1 = q[18:21]
    r2 = q[22:24]
    Q2 = q[25:28]
    r3 = q[29:31]
    Q3 = q[32:35]
    
    pb = rb + rotate(Qb, l_cb) # position vector from world frame to *JOINTS* 0 and 2
    
    c_ = [pb - r0 - rotate(Q0, -l_c0);  # ???
         [0 1 0 0; 0 0 0 1]*L(Qb)'*Q0;  # y axis rotation constraint
          r0 + rotate(Q0,  l0-l_c0) - r1 - rotate(Q1, -l_c1);
         [0 1 0 0; 0 0 0 1]*L(Q0)'*Q1;
          pb - r2 - rotate(Q2, -l_c2);
         [0 1 0 0; 0 0 0 1]*L(Qb)'*Q2;
          r2 + rotate(Q2, l2 - l_c2) - r3 - rotate(Q3, -l_c3);
         [0 1 0 0; 0 0 0 1]*L(Q2)'*Q3;
          r1 + rotate(Q1, l1 - l_c1) - r3 - rotate(Q3, lc-l_c3)]
         #[0 1 0 0; 0 0 0 1]*L(Q1)'*Q3]
end

c (generic function with 1 method)

In [40]:
function Dc(q)
    ForwardDiff.jacobian(dq->c(dq),q)*Ḡ(q)
end

Dc (generic function with 1 method)

In [41]:
#Initial Conditions
rb = zeros(3)
Qb = [1.0; 0; 0; 0]
q0 = -30*pi/180;
q1 = -120*(pi/180)
q2 = -150*(pi/180)
q3 = 120*(pi/180)

Tf = 10.0
h = 0.001 #20 Hz
thist = 0:h:Tf
N = length(thist)

pb = rb + rotate(Qb, l_cb)  # position vector from world frame to *JOINTS* 0 and 2

Qb0 = Expq([0, q0, 0])  # quaternion from base to link 0
Q0 = L(Qb)*Qb0  # quaternion from world frame to link 0
r0 = pb + rotate(Q0, l_c0)

Q01 = Expq([0, q1, 0])  # quaternion from link 0 to link 1
Q1 = L(Q0)*Q01  # quaternion from world frame to link 1
r1 = pb + rotate(Q0, l0) + rotate(Q1, l_c1)  

Qb2 = Expq([0, q2, 0])  # quaternion from base to link 2
Q2 = L(Qb)*Qb2  # quaternion from world frame to link 2
r2 = pb + rotate(Q2, l_c2)

Q23 = Expq([0, q3, 0])  # quaternion from base to link 2
Q3 = L(Q2)*Q23  # quaternion from world frame to link 2
r3 = pb + rotate(Q2, l2) + rotate(Q3, l_c3)  

rb_0 = rb
r0_0 = r0
r1_0 = r1
r2_0 = r2
r3_0 = r3
Qb_0 = Qb
Q0_0 = Q0
Q1_0 = Q1
Q2_0 = Q2
Q3_0 = Q3

q_0 = [rb_0; Qb_0; r0_0; Q0_0; r1_0; Q1_0; r2_0; Q2_0; r3_0; Q3_0]
# @show c(q_0)
#=
function qinv(Q)
    return reshape(Q, 1, 4)*Diagonal(T)/norm(Q)
end
=#

35-element Vector{Float64}:
  0.0
  0.0
  0.0
  1.0
  0.0
  0.0
  0.0
  0.010834702164954238
  0.00517191218927888
  0.006255418211525749
  0.9659258262890683
  0.0
 -0.25881904510252074
  ⋮
  0.023470645027595695
  0.25881904510252074
  0.0
 -0.9659258262890683
  0.0
 -0.15412702995699418
  0.004
  0.1933098469689183
  0.9659258262890683
  0.0
 -0.2588190451025209
  0.0

In [42]:
#Torque input at joints
uhist = repeat([1 0 0 0 0]*1e-4, N)'
@show size(uhist)
# 1 -> joint0
# 2 -> joint1
# 3 -> joint2
# 4 -> joint3
# 5 -> joint4 (parallel constraint)
#Corresponding F
Fhist = zeros(30,N)  # wrench [xyz force, xyz torque]
for k = 1:N
    Fhist[:,k] = [zeros(4); -uhist[1, k]-uhist[3, k]; 0; # body
                  zeros(4); uhist[1, k]-uhist[2, k]; 0; # link0
                  zeros(4); uhist[2, k]+uhist[5, k]; 0; # link1
                  zeros(4); uhist[3, k]-uhist[4, k]; 0; # link2
                  zeros(4); uhist[4, k]-uhist[5, k]; 0] # link3
end

size(uhist) = (5, 10001)


In [43]:
#Simulate
qhist = zeros(35,N)
qhist[:,1] .= q_0
qhist[:,2] .= q_0

@show n_c = size(c(q_0))[1]  # number of constraint rows

for k = 2:(N-1)
    
    #Initial guess
    qhist[:,k+1] .= qhist[:,k]
    λ = zeros(n_c)
    
    e = [DEL(qhist[:,k-1],qhist[:,k],qhist[:,k+1],λ,Fhist[:,k-1],Fhist[:,k]); c(qhist[:,k+1])]

    while maximum(abs.(e)) > 1e-12
        D = Dq3DEL(qhist[:,k-1],qhist[:,k],qhist[:,k+1],λ,Fhist[:,k-1],Fhist[:,k])
        C2 = Dc(qhist[:,k])
        C3 = Dc(qhist[:,k+1])

        Δ = -([D h*C2'; C3 zeros(n_c,n_c)]\e)
        
        qhist[1:3,k+1] .= qhist[1:3,k+1] + Δ[1:3]
        qhist[4:7,k+1] .= L(qhist[4:7,k+1])*[sqrt(1-Δ[4:6]'*Δ[4:6]); Δ[4:6]]
        qhist[8:10,k+1] .= qhist[8:10,k+1] + Δ[7:9]
        qhist[11:14,k+1] .= L(qhist[11:14,k+1])*[sqrt(1-Δ[10:12]'*Δ[10:12]); Δ[10:12]]
        qhist[15:17,k+1] .= qhist[15:17,k+1] + Δ[13:15]
        qhist[18:21,k+1] .= L(qhist[18:21,k+1])*[sqrt(1-Δ[16:18]'*Δ[16:18]); Δ[16:18]]
        qhist[22:24,k+1] .= qhist[22:24,k+1] + Δ[19:21]
        qhist[25:28,k+1] .= L(qhist[25:28,k+1])*[sqrt(1-Δ[22:24]'*Δ[22:24]); Δ[22:24]]
        qhist[29:31,k+1] .= qhist[29:31,k+1] + Δ[25:27]
        qhist[32:35,k+1] .= L(qhist[32:35,k+1])*[sqrt(1-Δ[28:30]'*Δ[28:30]); Δ[28:30]]

        λ .= λ + Δ[31:30+n_c]
        
        e = [DEL(qhist[:,k-1],qhist[:,k],qhist[:,k+1],λ,Fhist[:,k-1],Fhist[:,k]); c(qhist[:,k+1])]
        # print(maximum(abs.(e)), "\n") # 
        # print(c(qhist[:,k+1]), "\n")
        # flush(stdout)
    end
    print("\r Simulation ", trunc(Int, k/(N-1)*100), "% complete")
    flush(stdout)
end

n_c = (size(c(q_0)))[1] = 23
1.0000000000000001e-7
 Simulation 0% complete4.4887938520996456e-7
 Simulation 0% complete8.977587738320383e-7
 Simulation 0% complete1.3466381575516516e-6
 Simulation 0% complete1.7955175393540644e-6
 Simulation 0% complete2.2443969148105327e-6
 Simulation 0% complete2.693276279985592e-6
 Simulation 0% complete3.1421556487970354e-6
 Simulation 0% complete3.5910350070029045e-6
 Simulation 0% complete4.039914348101167e-6
 Simulation 0% complete4.488793684262038e-6
 Simulation 0% complete4.93767301183494e-6
 Simulation 0% complete5.38655232327188e-6
 Simulation 0% complete5.83543162041365e-6
 Simulation 0% complete6.28431090779977e-6
 Simulation 0% complete6.7331901722716245e-6
 Simulation 0% complete7.182069427749456e-6
 Simulation 0% complete7.63094865646157e-6
 Simulation 0% complete8.07982786832287e-6
 Simulation 0% complete8.528707059479586e-6
 Simulation 0% complete8.977586228562019e-6
 Simulation 0% complete9.426465379250055e-6
 Simulation 0% complete9

 Simulation 1% complete8.304147278582272e-5
 Simulation 1% complete8.349033222649576e-5
 Simulation 1% complete8.393919145121733e-5
 Simulation 1% complete8.438805045655886e-5
 Simulation 1% complete8.483690923890795e-5
 Simulation 1% complete8.528576781075896e-5
 Simulation 1% complete8.573462615958049e-5
 Simulation 1% complete8.618348427823374e-5
 Simulation 1% complete8.663234218033072e-5
 Simulation 1% complete8.708119985618606e-5
 Simulation 1% complete8.753005731218256e-5
 Simulation 1% complete8.797891452903836e-5
 Simulation 1% complete8.842777152504514e-5
 Simulation 1% complete8.887662829142172e-5
 Simulation 2% complete8.932548482796056e-5
 Simulation 2% complete8.977434113223662e-5
 Simulation 2% complete9.022319719662788e-5
 Simulation 2% complete9.067205303849806e-5
 Simulation 2% complete9.112090863293107e-5
 Simulation 2% complete9.156976400165014e-5
 Simulation 2% complete9.201861912694315e-5
 Simulation 2% complete9.246747401559909e-5
 Simulation 2% complete9.2916328

 Simulation 3% complete0.00016428034312825584
 Simulation 3% complete0.00016472914381059425
 Simulation 3% complete0.0001651779440546973
 Simulation 3% complete0.00016562674386038712
 Simulation 3% complete0.00016607554323501002
 Simulation 3% complete0.00016652434216363025
 Simulation 3% complete0.00016697314064971052
 Simulation 3% complete0.00016742193869948832
 Simulation 3% complete0.00016787073629942134
 Simulation 3% complete0.00016831953345415213
 Simulation 3% complete0.00016876833016519861
 Simulation 3% complete0.00016921712642777587
 Simulation 3% complete0.00016966592224253613
 Simulation 3% complete0.00017011471760511546
 Simulation 3% complete0.00017056351251902991
 Simulation 3% complete0.0001710123069768798
 Simulation 3% complete0.00017146110098343226
 Simulation 3% complete0.0001719098945330426
 Simulation 3% complete0.00017235868762505862
 Simulation 3% complete0.00017280748026488784
 Simulation 3% complete0.00017325627243624852
 Simulation 3% complete0.000173705064

 Simulation 5% complete0.0002446075064311974
 Simulation 5% complete0.0002450562083843098
 Simulation 5% complete0.0002455049096585134
 Simulation 5% complete0.000245953610260366
 Simulation 5% complete0.0002464023101949311
 Simulation 5% complete0.00024685100944704175
 Simulation 5% complete0.00024729970802155987
 Simulation 5% complete0.0002477484059160783
 Simulation 5% complete0.00024819710313510324
 Simulation 5% complete0.00024864579966674057
 Simulation 5% complete0.0002490944955211649
 Simulation 5% complete0.00024954319068631615
 Simulation 5% complete0.00024999188516817105
 Simulation 5% complete0.000250440578960907
 Simulation 5% complete0.00025088927206766654
 Simulation 5% complete0.00025133796448237815
 Simulation 5% complete0.0002517866562099631
 Simulation 5% complete0.0002522353472378158
 Simulation 5% complete0.0002526840375785419
 Simulation 5% complete0.000253132727219844
 Simulation 5% complete0.00025358141616533907
 Simulation 5% complete0.000254030104413272
 Simu

 Simulation 7% complete0.00032536167121362584
 Simulation 7% complete0.0003258102290329759
 Simulation 7% complete0.0003262587859174049
 Simulation 7% complete0.00032670734186534753
 Simulation 7% complete0.0003271558968735308
 Simulation 7% complete0.0003276044509389664
 Simulation 7% complete0.0003280530040663029
 Simulation 7% complete0.00032850155624282787
 Simulation 7% complete0.0003289501074788346
 Simulation 7% complete0.00032939865776265435
 Simulation 7% complete0.00032984720710166305
 Simulation 7% complete0.000330295755484192
 Simulation 7% complete0.0003307443029183286
 Simulation 7% complete0.00033119284939961413
 Simulation 7% complete0.0003316413949251315
 Simulation 7% complete0.00033208993949210557
 Simulation 7% complete0.00033253848310433134
 Simulation 7% complete0.0003329870257535314
 Simulation 7% complete0.00033343556744845734
 Simulation 7% complete0.00033388410816866527
 Simulation 7% complete0.0003343326479266776
 Simulation 7% complete0.0003347811867247238
 

...

 Simulation 91% complete0.003198530964452573
 Simulation 91% complete0.0032018550120450906
 Simulation 91% complete0.0032051773673444925
 Simulation 91% complete0.003208498029182768
 Simulation 91% complete0.0032118169963592713
 Simulation 91% complete0.0032151342677598765
 Simulation 91% complete0.0032184498421513035
 Simulation 91% complete0.003221763718411078
 Simulation 91% complete0.003225075895428868
 Simulation 91% complete0.003228386372041976
 Simulation 91% complete0.003231695147122614
 Simulation 91% complete0.0032350022195809427
 Simulation 91% complete0.003238307588294487
 Simulation 91% complete0.0032416112521309066
 Simulation 91% complete0.003244913210091434
 Simulation 91% complete0.0032482134610239966
 Simulation 91% complete0.003251512003902505
 Simulation 91% complete0.0032548088376302886
 Simulation 91% complete0.003258103961202509
 Simulation 91% complete0.003261397373520978
 Simulation 91% complete0.003264689073596036
 Simulation 91% complete0.0032679790603970396


 Simulation 93% complete0.0037688196785304762
 Simulation 93% complete0.0037718278718118633
 Simulation 93% complete0.0037748342863824997
 Simulation 93% complete0.0037778389223326993
 Simulation 93% complete0.0037808417798430903
 Simulation 93% complete0.003783842858976665
 Simulation 93% complete0.0037868421598844524
 Simulation 93% complete0.0037898396826886423
 Simulation 93% complete0.0037928354275455767
 Simulation 93% complete0.0037958293946115974
 Simulation 93% complete0.0037988215839648756
 Simulation 93% complete0.0038018119958353703
 Simulation 93% complete0.003804800630349066
 Simulation 93% complete0.003807787487684314
 Simulation 93% complete0.0038107725680020096
 Simulation 93% complete0.003813755871443316
 Simulation 93% complete0.0038167373982093532
 Simulation 93% complete0.0038197171485080707
 Simulation 93% complete0.0038226951224958105
 Simulation 93% complete0.0038256713203949425
 Simulation 93% complete0.0038286457423800232
 Simulation 93% complete0.003831618388

 Simulation 95% complete0.004284633690354329
 Simulation 95% complete0.004287326040389194
 Simulation 95% complete0.004290016699460609
 Simulation 95% complete0.004292705668357873
 Simulation 95% complete0.004295392947820196
 Simulation 95% complete0.004298078538661162
 Simulation 95% complete0.004300762441654133
 Simulation 95% complete0.0043034446575588095
 Simulation 95% complete0.004306125187167526
 Simulation 95% complete0.004308804031277931
 Simulation 95% complete0.004311481190664903
 Simulation 95% complete0.004314156666132161
 Simulation 95% complete0.004316830458434852
 Simulation 95% complete0.004319502568416161
 Simulation 95% complete0.004322172996851724
 Simulation 95% complete0.004324841744523253
 Simulation 95% complete0.004327508812258752
 Simulation 95% complete0.004330174200850557
 Simulation 95% complete0.00433283791109176
 Simulation 95% complete0.004335499943805055
 Simulation 95% complete0.004338160299782777
 Simulation 95% complete0.004340818979855967
 Simulatio

 Simulation 97% complete0.004747634884467522
 Simulation 97% complete0.00475003394142799
 Simulation 97% complete0.004752431474523439
 Simulation 97% complete0.004754827484780718
 Simulation 97% complete0.004757221973196316
 Simulation 97% complete0.004759614940781144
 Simulation 97% complete0.004762006388565844
 Simulation 97% complete0.004764396317555254
 Simulation 97% complete0.004766784728770152
 Simulation 97% complete0.00476917162321993
 Simulation 97% complete0.004771557001937506
 Simulation 97% complete0.004773940865948211
 Simulation 97% complete0.004776323216240945
 Simulation 97% complete0.0047787040538380034
 Simulation 97% complete0.004781083379781412
 Simulation 97% complete0.004783461195080564
 Simulation 97% complete0.004785837500753199
 Simulation 97% complete0.004788212297826165
 Simulation 97% complete0.004790585587308854
 Simulation 97% complete0.004792957370252401
 Simulation 97% complete0.004795327647622179
 Simulation 97% complete0.0047976964204920905
 Simulatio

 Simulation 98% complete0.005160181035035742
 Simulation 98% complete0.005162319746122987
 Simulation 98% complete0.005164457121620389
 Simulation 98% complete0.005166593162538101
 Simulation 98% complete0.005168727869893862
 Simulation 98% complete0.005170861244838988
 Simulation 98% complete0.005172993288237913
 Simulation 98% complete0.0051751240012077974
 Simulation 98% complete0.005177253384772455
 Simulation 98% complete0.005179381439939
 Simulation 98% complete0.005181508167755532
 Simulation 98% complete0.005183633569223094
 Simulation 98% complete0.005185757645403065
 Simulation 98% complete0.005187880397300665
 Simulation 99% complete0.0051900018259495705
 Simulation 99% complete0.0051921219323808054
 Simulation 99% complete0.005194240717611728
 Simulation 99% complete0.005196358182672984
 Simulation 99% complete0.005198474328594834
 Simulation 99% complete0.005200589156385913
 Simulation 99% complete0.005202702667087109
 Simulation 99% complete0.0052048148617264076
 Simulati

In [44]:
qhist

35×10001 Matrix{Float64}:
  0.0          0.0         -6.41256e-11  …  -0.00169341   -0.00169323
  0.0          0.0          1.13022e-12      5.1166e-6     5.09954e-6
  0.0          0.0          3.60383e-11      0.000411335   0.000410565
  1.0          1.0          1.0              0.844105      0.844045
  0.0          0.0         -7.67293e-11      0.00202217    0.00202275
  0.0          0.0         -1.13183e-8   …  -0.536016     -0.536112
  0.0          0.0          2.39214e-10      0.0129929     0.012995
  0.0108347    0.0108347    0.0108347       -0.00565472   -0.00566023
  0.00517191   0.00517191   0.00517191       0.0049575     0.00495731
  0.00625542   0.00625542   0.00625542      -0.0115491    -0.011548
  0.965926     0.965926     0.965926     …   0.588644      0.588452
  0.0          0.0         -1.22018e-11     -0.0128379    -0.012841
 -0.258819    -0.258819    -0.258819         0.808285      0.808425
  ⋮                                      ⋱                 ⋮
  0.0234706    0

In [16]:



#=
q0 = -xhist[8, :] .- 30*(pi/180)
q1 = -xhist[9, :] .- 120*(pi/180)
q2 = -xhist[10, :] .- 150*(pi/180)
q3 = -xhist[11, :] .+ 120*(pi/180)

qs = convert(AbstractArray{Float64}, [q0 q2 q1 q3]) 
ts = convert(AbstractVector{Float64}, thist) # AbstractVector(thist)
q_array = [ qs[i,:] for i in 1:size(qs,1)] 
=#

LoadError: UndefVarError: xhist not defined

In [17]:
using MeshCat, GeometryBasics, CoordinateTransformations, Rotations#, GeometryTypes
#const gt = GeometryTypes

In [18]:
vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/ben/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8700))

In [54]:
delete!(vis)

green_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.8))
red_material = MeshPhongMaterial(color=RGBA(1, 0, 0, 0.8))

#l_345 = sqrt((l3 + l4)^2 + l5^2)
# sphere = Sphere(Point(0, 0, 0), 3)
# rect = Rectangle(0.2, 0.2, 0.2, 0.2)
cylinderb = Cylinder(Point(0, -0.02, l_cb[2]), Point(0, 0.02, l_cb[2]), 0.04)  #float(l_cb)
cylinder0 = Cylinder(Point(-l_c0[1], -l_c0[2], -l_c0[3]), Point((l0-l_c0)[1], (l0-l_c0)[2], (l0-l_c0)[3]), 0.008)
cylinder1 = Cylinder(Point(-l_c1[1], -l_c1[2], -l_c1[3]), Point((l1-l_c1)[1],(l1-l_c1)[2],(l1-l_c1)[2]), 0.008)
cylinder2 = Cylinder(Point(-l_c2[1], -l_c2[2], -l_c2[3]), Point((l2-l_c2)[1], (l2-l_c2)[2], (l2-l_c2)[3]), 0.008)
cylinder3 = Cylinder(Point(-l_c3[1], -l_c3[2], -l_c3[3]), Point((lee-l_c3)[1], (lee-l_c3)[2],  (lee-l_c3)[3]), 0.008)

# setobject!(vis["sphere"],sphere,red_material)
setobject!(vis["cylinderb"],cylinderb,red_material)
setobject!(vis["cylinder0"],cylinder0,green_material)
setobject!(vis["cylinder1"],cylinder1,green_material)
setobject!(vis["cylinder2"],cylinder2,green_material)
setobject!(vis["cylinder3"],cylinder3,green_material)

for k = 1:N
    
    q = qhist[:, k]
    # set position and attitude
    positionb = Translation(q[1:3]...)
    attitudeb = LinearMap(UnitQuaternion(q[4:7]))
    position0 = Translation(q[8:10]...)
    attitude0 = LinearMap(UnitQuaternion(q[11:14]))
    position1 = Translation(q[15:17]...)
    attitude1 = LinearMap(UnitQuaternion(q[18:21]))
    position2 = Translation(q[22:24]...)
    attitude2 = LinearMap(UnitQuaternion(q[25:28]))
    position3 = Translation(q[29:31]...)
    attitude3 = LinearMap(UnitQuaternion(q[32:35]))
    
    settransform!(vis["cylinderb"], compose(positionb,attitudeb))
    settransform!(vis["cylinder0"], compose(position0,attitude0))
    settransform!(vis["cylinder1"], compose(position1,attitude1))
    settransform!(vis["cylinder2"], compose(position2,attitude2))
    settransform!(vis["cylinder3"], compose(position3,attitude3))
    #sleep(0.001)
end

In [21]:
# only works with Rotations v1.0.4 and earlier
# use Pkg.add(Pkg.PackageSpec(;name="Rotations", version="1.0.4"))
curdir = pwd()
urdfpath = joinpath(curdir, "../res/flyhopper_robot/urdf/flyhopper_robot.urdf")
leg = parse_urdf(urdfpath, floating=true)
state = MechanismState(leg)

MechanismState{Float64, Float64, Float64, …}(…)

In [22]:
# mvis = MechanismVisualizer(doublependulum, Skeleton(randomize_colors=true, inertias=false));
mvis = MechanismVisualizer(leg, URDFVisuals(urdfpath));

render(mvis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8701
└ @ MeshCat /home/ben/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Link 2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x69, 0x6e, 0x6b, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("Link 3" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6b, 0x20, 0x32, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 3" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_5" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x35], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x35], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), "after_Joint 2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), "Link 0" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x69, 0x6e, 0x6b, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x30], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("Link 1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6b, 0x20, 0x30, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x31], Dict{String, Vector{UInt8}}(), nothing

In [23]:
q_0 = x_init[8]
q_1 = x_init[9]
q_2 = x_init[10]
q_3 = x_init[11]

set_configuration!(mvis, [1,0,0,0,0,0,0, -q_0-30*(pi/180), -q_2-150*(pi/180), -q_1-120*(pi/180), -q_3+120*(pi/180)])

LoadError: UndefVarError: x_init not defined

In [24]:
for k = 1:N
    pb = xhist[1:3, k]
    Qb = xhist[4:7, k]
    input = vcat(Qb, pb, -q_array[k])
    set_configuration!(mvis, input)
end

LoadError: UndefVarError: xhist not defined

In [25]:
#Energy Functions

function Energy(x)
    q = x[1:11]
    v = x[12:end]
    energy = Lagrangian(q,v)
    return energy
end

Energy (generic function with 1 method)

In [26]:
#Plot total energy
using Plots

E = zeros(N)
for k = 1:N
    E[k] = Energy(xhist[:, k])
end

plot(thist,E, title="Total Energy vs Time", xlabel="Time (s)", ylabel="Energy (Joules)")

LoadError: UndefVarError: xhist not defined